In [38]:
# https://wikidocs.net/55409

---
# 04. nn.Module로 구현하는 선형 회귀

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [40]:
torch.manual_seed(1)

In [41]:
# y = 2x를 가정한 데이터 생성 / w=2 , b=0

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])
print(x_train.size())
print(y_train.size())

torch.Size([3, 1])
torch.Size([3, 1])


In [42]:
# 단순 선형 회귀를 위한 모델 선언 및 초기화
# linear는 (input_dim, output_dim)을 인수로 받는다.
# 여기서는 input_dim=1 , output_dim=1
model = nn.Linear(1,1)

In [43]:
# 불러온 모델에는 가중치w 와 편향b가 저장되어 있는데, model.parameters()에서 확인 할 수 있다.
print(list(model.parameters()))

# [Parameter containing:
# tensor([[0.5153]], requires_grad=True), Parameter containing:   --> W(랜덤시드 1에 생성된 값)
# tensor([-0.4414], requires_grad=True)]                          --> b
# 두 값 모두 학습의 대상이므로 requires_grad=True

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


In [44]:
# 옵티마이저 정의
# 경사 하강법 SGD를 사용하고 lr = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [45]:
# 에폭 2000으로 모델 설정
nb_epochs = 2000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/2000 Cost: 13.103541
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


In [46]:
# 테스트 데이터를 넣어서 확인 해보자

# x = 4
new_var = torch.FloatTensor([[4.0]])
pred_y = model(new_var) # forward 계산
# - H(x)에 입력 x로부터 예측된 y를 얻는 것을 forward 연산이라 한다.
# - 학습 과정에서 비용 함수를 미분하여 기울기를 구하는 것을 backward 연산이라고 합니다.
# - cost.backward()는 비용 함수로부터 기울기를 구하라는 의미이며 backward 연산입니다.
print('4가 입력일 때 y의 값(8): ', pred_y)

4가 입력일 때 y의 값(8):  tensor([[7.9989]], grad_fn=<AddmmBackward>)


In [47]:
# cost가 감소 했음을 확인. 다음은 W와 b가 최적화 되었는지 확인
# x에 임의의 값 4를 넣어 모델이 예측하는 y의 값을 확인하자

print(list(model.parameters()))

[Parameter containing:
tensor([[1.9994]], requires_grad=True), Parameter containing:
tensor([0.0014], requires_grad=True)]


---
## 다중 선형 회귀 구현

In [48]:
# 3개의 x로부터 하나의 y를 예측하는 문제를 만들자
# hypothesis = H(x) = w1x1 + w2x2 + w3x3 + b

# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

print(x_train.size())
print(y_train.size())

torch.Size([5, 3])
torch.Size([5, 1])


In [49]:
# 다중 선형 회귀이므로 input_dim=3, output_dim=1
model = nn.Linear(3,1)

In [50]:
# 초기값으로 저장된 w,b를 확인 (랜덤 시드 1로 설정함)
print('W: ', list(model.parameters())[0])
print('b: ', list(model.parameters())[1])

W:  Parameter containing:
tensor([[-0.1119,  0.2710, -0.5435]], requires_grad=True)
b:  Parameter containing:
tensor([0.3462], requires_grad=True)


In [51]:
# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [52]:
nb_epochs = 2000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/2000 Cost: 42134.7
Epoch  100/2000 Cost: 5.96005
Epoch  200/2000 Cost: 5.65471
Epoch  300/2000 Cost: 5.36541
Epoch  400/2000 Cost: 5.09143
Epoch  500/2000 Cost: 4.83183
Epoch  600/2000 Cost: 4.586
Epoch  700/2000 Cost: 4.35307
Epoch  800/2000 Cost: 4.13241
Epoch  900/2000 Cost: 3.92346
Epoch 1000/2000 Cost: 3.7255
Epoch 1100/2000 Cost: 3.53797
Epoch 1200/2000 Cost: 3.36033
Epoch 1300/2000 Cost: 3.19206
Epoch 1400/2000 Cost: 3.03267
Epoch 1500/2000 Cost: 2.8817
Epoch 1600/2000 Cost: 2.73867
Epoch 1700/2000 Cost: 2.6032
Epoch 1800/2000 Cost: 2.47485
Epoch 1900/2000 Cost: 2.35329
Epoch 2000/2000 Cost: 2.23811


In [54]:
# 예측하여 성능 평가
new_var =  torch.FloatTensor([[73, 80, 75]]) 
pred_y = model(new_var)
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[153.7184]], grad_fn=<AddmmBackward>)


In [55]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.8541, 0.8475, 0.3096]], requires_grad=True), Parameter containing:
tensor([0.3568], requires_grad=True)]
